Teste

In [1]:
print ("hello, world!")

hello, world!


In [2]:
%pip install --upgrade pip
%pip install python-constraint

from constraint import *


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


# Instanciar o Problema

In [3]:

days = 'Segunda Terça Quarta Quinta Sexta'.split()
class_groups = 't01 t02 t03'.split()
blocks = 'B01 B02 B03 B04 B05 B06 B07 B08 B09 B10 B11 B12 B13 B14 B15 B16 B17 B18 B19 B20'.split()
lecturers = 'jo mike rob sue'.split()
courses = 'UC11 UC12 UC13 UC14 UC15 UC21 UC22 UC23 UC24 UC25 UC31 UC32 UC33 UC34 UC35'.split()
rooms = 'A C G Lab01 Lab02'.split()


# Constrains

In [4]:

course_classes = {
    't01': ['UC11', 'UC12', 'UC13', 'UC14', 'UC15'],
    't02': ['UC21', 'UC22', 'UC23', 'UC24', 'UC25'],
    't03': ['UC31', 'UC32', 'UC33', 'UC34', 'UC35']
}
class_lecturers = {
    'jo' : ['UC11', 'UC21', 'UC22', 'UC31'],
    'mike': ['UC12', 'UC23', 'UC32'],
    'rob': ['UC13', 'UC14', 'UC24', 'UC33'],
    'sue': ['UC15', 'UC25', 'UC34', 'UC35']
}
class_rooms = {
    'UC22' : ['Lab01'],
    'UC14' : ['Lab01'],
}

time_restrictions = {
    'mike' : ['B13', 'B14', 'B15', 'B16', 'B17', 'B18', 'B19', 'B20'],
    'rob' : ['B01', 'B02', 'B03', 'B04'],
    'sue' : ['B09', 'B10', 'B11', 'B12', 'B17', 'B18', 'B19', 'B20']
    }

online_lessons = {'UC21': {2}, 'UC31': {2}}

days_blocks = {
    'Segunda': ['B01', 'B02', 'B03', 'B04'],
    'Terça': ['B05', 'B06', 'B07', 'B08'],
    'Quarta': ['B09', 'B10', 'B11', 'B12'],
    'Quinta': ['B13', 'B14', 'B15', 'B16'],
    'Sexta': ['B17', 'B18', 'B19', 'B20'],
}

online_lesson = {"UC21_2", "UC31_2"}

physical_rooms = [r for r in rooms if r != "Online"]

# Min Conflicts Solver (teste)

In [17]:
from itertools import combinations

# ——— 1) Dados auxiliares ———
online_lessons_set = {"UC21_2", "UC31_2"}

# Aulas a agendar
lessons_to_schedule = [f"{course_code}_{index}" for course_code in courses for index in (1, 2)]

# Inversões (UC -> docente; UC -> turma)
course_to_teacher = {course_code: lecturer_name for lecturer_name, course_list in class_lecturers.items() for course_code in course_list}
course_to_group = {course_code: group_name for group_name, course_list in course_classes.items() for course_code in course_list}

# ——— 2) Domínio ———
def domain_for_lesson(lesson_name):
    course_code = lesson_name.split("_")[0]
    teacher_name = course_to_teacher[course_code]
    teacher_blocked_blocks = set(time_restrictions.get(teacher_name, []))
    allowed_blocks = [block for block in blocks if block not in teacher_blocked_blocks]

    if lesson_name in online_lessons_set:
        candidate_rooms = ["Online"]                 # aula online não ocupa sala física
    elif course_code in class_rooms and class_rooms[course_code]:
        candidate_rooms = class_rooms[course_code]   # salas obrigatórias para a UC
    else:
        candidate_rooms = physical_rooms             # qualquer sala física

    return [(block, room) for block in allowed_blocks for room in candidate_rooms]

# ——— 3) Criar o problema ———
tt_problem = Problem(solver=MinConflictsSolver())


# Minimum Remaining Values - Para fazer aulas com menos opções primeiro e tornar o algoritmo mais eficiente e tal
lesson_domains = {lesson_name: domain_for_lesson(lesson_name) for lesson_name in lessons_to_schedule}
for lesson_name in sorted(lessons_to_schedule, key=lambda name: len(lesson_domains[name])):
    tt_problem.addVariable(lesson_name, lesson_domains[lesson_name])

# ——— 4) Constraints ———
def no_overlapping_blocks(*assigned_blocks_rooms):
    # Impede choques de bloco
    return len({block for (block, room) in assigned_blocks_rooms}) == len(assigned_blocks_rooms)

def not_same_physical_room_in_block(lesson_assignment_left, lesson_assignment_right):
    # Uma aula por (sala, bloco), exceto a aula online
    (block_left, room_left) = lesson_assignment_left
    (block_right, room_right) = lesson_assignment_right
    if room_left == "Online" or room_right == "Online":
        return True
    return not (block_left == block_right and room_left == room_right)

block_to_day_map = {block: day for day, day_blocks in days_blocks.items() for block in day_blocks}
def lessons_on_different_days(lesson_assignment_first, lesson_assignment_second):
    # As 2 aulas da mesma UC em dias diferentes
    return block_to_day_map[lesson_assignment_first[0]] != block_to_day_map[lesson_assignment_second[0]]

# 4a) Por docente
for lecturer_name in lecturers:
    variables_for_lecturer = [f"{course_code}_{index}" for course_code in class_lecturers[lecturer_name] for index in (1, 2)]
    tt_problem.addConstraint(no_overlapping_blocks, tuple(variables_for_lecturer))

# 4b) Por turma
for group_name in class_groups:
    variables_for_group = [f"{course_code}_{index}" for course_code in course_classes[group_name] for index in (1, 2)]
    tt_problem.addConstraint(no_overlapping_blocks, tuple(variables_for_group))

# 4c) Por sala (binárias por par)
for left_lesson, right_lesson in combinations(lessons_to_schedule, 2):
    tt_problem.addConstraint(not_same_physical_room_in_block, (left_lesson, right_lesson))

# 4d) Duas aulas da mesma UC em dias distintos
for course_code in courses:
    tt_problem.addConstraint(lessons_on_different_days, (f"{course_code}_1", f"{course_code}_2"))

# ——— 5) Resolver ———

solution = tt_problem.getSolution()

# todo: ordenar a resposta por blocos, fazer as soft e o resto das hard restrictions, dormir

print(solution if solution else "Sem solução encontrada.")

{'UC14_1': ('B09', 'Lab01'), 'UC14_2': ('B17', 'Lab01'), 'UC21_2': ('B18', 'Online'), 'UC22_1': ('B20', 'Lab01'), 'UC22_2': ('B16', 'Lab01'), 'UC31_2': ('B17', 'Online'), 'UC12_1': ('B07', 'A'), 'UC12_2': ('B12', 'Lab01'), 'UC15_1': ('B16', 'G'), 'UC15_2': ('B06', 'G'), 'UC23_1': ('B03', 'C'), 'UC23_2': ('B09', 'G'), 'UC25_1': ('B01', 'A'), 'UC25_2': ('B14', 'G'), 'UC32_1': ('B01', 'C'), 'UC32_2': ('B10', 'Lab02'), 'UC34_1': ('B05', 'Lab01'), 'UC34_2': ('B03', 'G'), 'UC35_1': ('B08', 'C'), 'UC35_2': ('B02', 'C'), 'UC13_1': ('B15', 'Lab01'), 'UC13_2': ('B11', 'A'), 'UC24_1': ('B12', 'C'), 'UC24_2': ('B05', 'Lab02'), 'UC33_1': ('B19', 'C'), 'UC33_2': ('B07', 'Lab02'), 'UC11_1': ('B01', 'Lab02'), 'UC11_2': ('B19', 'A'), 'UC21_1': ('B08', 'Lab01'), 'UC31_1': ('B04', 'C')}
